In [0]:
import requests
import json
from pyspark.sql import Row

# Parámetros API OpenSky
url = "https://opensky-network.org/api/states/all"
resp = requests.get(url, timeout=30)
resp.raise_for_status()
data = resp.json()

# Convertir a DataFrame Spark
fields = [
  "icao24", "callsign", "origin_country",
  "time_position", "last_contact",
  "longitude", "latitude", "baro_altitude",
  "on_ground", "velocity", "true_track",
  "vertical_rate", "sensors", "geo_altitude",
  "squawk", "spi", "position_source"
]
rows = [Row(**dict(zip(fields, state))) for state in data.get("states", [])]
df = spark.createDataFrame(rows)

# Guardar en Bronze como Delta
bronze_path = "/mnt/datalake/bronze/opensky/"
df.write.format("delta").mode("append").save(bronze_path)
print(f"✅ Ingesta completada: {df.count()} registros insertados en Bronze")
